# Get all the data from users on Admin

## Scraping on a site with login

In [1]:
# Prepare a function to read the configuration file with the confidential data...
# ...because the website is an private website on an Intranet, the production file is config.yaml.
# In GitHub there is only a config.yaml.example with dummy data

import yaml

__config = None

def config():

    global __config

    if not __config:
        with open('./scraper_ETL/extract/config.yaml', mode='r') as f:
            config = yaml.safe_load(f)

    return config


In [2]:
# Get the url on the config file
host = config()['websites']['admin_users']['url']
#host

In [3]:
#Import selenium
from selenium import webdriver

# Setup driver
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(executable_path='./driver/chromedriver', options=options)

# Get the site
driver.get(host)


In [4]:
# Setup login
# Find the login elements and ensure that there are only one of each type

driver.find_elements_by_xpath('//input[@name="usuario"]')


[<selenium.webdriver.remote.webelement.WebElement (session="ec1051b766e184492fafa05d1b20a6ec", element="207d6171-a668-49cd-ba25-7c80f0146912")>]

In [5]:
driver.find_elements_by_xpath('//input[@name="pwd"]')

[<selenium.webdriver.remote.webelement.WebElement (session="ec1051b766e184492fafa05d1b20a6ec", element="fc6466aa-2553-4550-b37d-0e36c5e77e41")>]

In [6]:
# Get the credential data
user = config()['websites']['admin_users']['user']
password = config()['websites']['admin_users']['password']

alt_bntAceptarLogin = config()['websites']['admin_users']['labels']['page01']['alt_bntAceptarLogin']


In [7]:
# Login
driver.find_element_by_xpath('//input[@name="usuario"]').send_keys(user)
driver.find_element_by_xpath('//input[@name="pwd"]').send_keys(password)

# Click to login... the element is an input without name and an alt tag defined on config.yaml: 
    # <input type="image" name src="bt_aceptar.gif" alt="Entar al Sistema">
driver.find_element_by_xpath('//input[@alt="' + alt_bntAceptarLogin + '"]').click()

### Successful login!

## Navigate to the important data

In [8]:
# Locate the menu option
# <a href="javascript:catchSelection(4000);">Control de Usuarios</a>
text_menuCtrlUsuarios = config()['websites']['admin_users']['labels']['page02']['text_menuCtrlUsuarios']

driver.find_element_by_xpath('//a[text()="' + text_menuCtrlUsuarios + '"]')

<selenium.webdriver.remote.webelement.WebElement (session="ec1051b766e184492fafa05d1b20a6ec", element="7f1b28a6-e161-47ff-b0b6-20c91366f9d9")>

In [9]:
driver.find_element_by_xpath('//a[text()="' + text_menuCtrlUsuarios + '"]').click()

In [10]:
# Now, the next element is a select menu, so we need to import Select from Selenium
from selenium.webdriver.support.select import Select


In [11]:
# Inspect the select and print the options

# First we get the name of the select menu
name_selectMenu = config()['websites']['admin_users']['labels']['page03']['name_selectMenu']

test = Select(driver.find_element_by_name(name_selectMenu))
# test.select_by_value("Consulta de Usuarios")
print(test.options[0].text)
print(test.options[1].text)
print(test.options[2].text)
print(test.options[3].text)
print(test.options[4].text)

Seleccione
Alta de Usuarios
Baja de Usuarios
Modificar Usuarios
Consulta de Usuarios


In [12]:
# Select the correct option
test = Select(driver.find_element_by_name(name_selectMenu))
option_selectMenu = config()['websites']['admin_users']['labels']['page03']['option_selectMenu']
test.select_by_value(option_selectMenu)

In [13]:
# Now, we had selected the option on the menu, click on button "Aceptar"
alt_bntAceptarSelect = config()['websites']['admin_users']['labels']['page03']['alt_bntAceptarSelect']
driver.find_element_by_xpath('//img[@alt="' + alt_bntAceptarSelect + '"]').click()

In [14]:
# Query all the accounts

# <img src="bt_consulta.gif" alt="Consultar" ... >
# Click on button
alt_bntConsulta = config()['websites']['admin_users']['labels']['page04']['alt_bntConsulta']
driver.find_element_by_xpath('//img[@alt="' + alt_bntConsulta + '"]').click()

## Get the data

In [15]:
# Inspect pagination

# Find element
# rows = driver.find_element_by_xpath('//table[@class="txt"]').get_attribute('cellspacing')
# rows = driver.find_elements_by_xpath('//table[@class="txt"]/tbody/tr/td/table/tbody/tr') //26 rows (max 13 rows)
# Select rows of data without the titles row (not @valign) and even rows // 65 data

class_table = config()['websites']['admin_users']['labels']['page05']['class_table']
class_td = config()['websites']['admin_users']['labels']['page05']['class_td']
even_rows = driver.find_elements_by_xpath('//table[@class="' + class_table + '"]/tbody/tr/td/table/tbody/tr[not(@valign)]/td[@class="txtb"]')

len(even_rows)

65

In [16]:
# Select rows of data without the titles row (not @valign) and even rows // 60 data (max 12 rows)
odd_rows = driver.find_elements_by_xpath('//table[@class="' + class_table + '"]/tbody/tr/td/table/tbody/tr[not(@valign)]/td[@class="txt"]')

len(odd_rows)


60

In [ ]:
# Testing with the first and second rows
i = 0
j = 0

curp = even_rows[i].text
print('CURP:', curp)
i = i + 1
matricula = even_rows[i].text
print('Matrícula:', matricula)
i = i + 1
nombre = even_rows[i].text
print('Nombre:', nombre)
i = i + 1
cuenta = even_rows[i].text
print('Cuenta:', cuenta)
i = i + 1
grupo = even_rows[i].text
print('Grupo:', grupo)
i = i + 1

curp = odd_rows[j].text
print('CURP:', curp)
j = j + 1
matricula = odd_rows[j].text
print('Matrícula:', matricula)
j = j + 1
nombre = odd_rows[j].text
print('Nombre:', nombre)
j = j + 1
cuenta = odd_rows[j].text
print('Cuenta:', cuenta)
j = j + 1
grupo = odd_rows[j].text
print('Grupo:', grupo)
j = j + 1

In [17]:
# Or, even better, select all the data in the same object with:

class_table = config()['websites']['admin_users']['labels']['page05']['class_table']
class_td = config()['websites']['admin_users']['labels']['page05']['class_td']

all_rows = driver.find_elements_by_xpath('//table[@class="' + class_table + '"]/tbody/tr/td/table/tbody/tr[not(@valign)]/td[contains(@class, "' + class_td + '")]')

len(all_rows)

# //table[@class="txt"]/tbody/tr/td/table/tbody/tr[not(@valign)]/td[contains(@class, "txt")]



125

In [ ]:
# Testing with the first and second rows
i = 0

print("Record #", int(i / 5 + 1))
curp = all_rows[i].text
print('CURP:', curp)
i = i + 1
matricula = all_rows[i].text
print('Matrícula:', matricula)
i = i + 1
nombre = all_rows[i].text
print('Nombre:', nombre)
i = i + 1
cuenta = all_rows[i].text
print('Cuenta:', cuenta)
i = i + 1
grupo = all_rows[i].text
print('Grupo:', grupo)
i = i + 1
print("")

print("Record #", int(i / 5 + 1))
curp = all_rows[i].text
print('CURP:', curp)
i = i + 1
matricula = all_rows[i].text
print('Matrícula:', matricula)
i = i + 1
nombre = all_rows[i].text
print('Nombre:', nombre)
i = i + 1
cuenta = all_rows[i].text
print('Cuenta:', cuenta)
i = i + 1
grupo = all_rows[i].text
print('Grupo:', grupo)
i = i + 1


In [ ]:
# Testing with another row
i=120

print("Last Record #", int(i / 5 + 1))
curp = all_rows[i].text
print('CURP:', curp)
i = i + 1
matricula = all_rows[i].text
print('Matrícula:', matricula)
i = i + 1
nombre = all_rows[i].text
print('Nombre:', nombre)
i = i + 1
cuenta = all_rows[i].text
print('Cuenta:', cuenta)
i = i + 1
grupo = all_rows[i].text
print('Grupo:', grupo)
i = i + 1

### Successful scrapper on page data!

## Change to the next page

In [18]:
# Count the pages...
class_td = config()['websites']['admin_users']['labels']['page05']['class_td']
class_a = config()['websites']['admin_users']['labels']['page05']['class_a']
pages = driver.find_element_by_xpath('//td[@class="' + class_td + '"]').find_elements_by_xpath('//a[@class="' + class_a + '"]')
pages

[<selenium.webdriver.remote.webelement.WebElement (session="ec1051b766e184492fafa05d1b20a6ec", element="8e4b22d7-31ee-4263-904c-b7a4e5d29ff6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="ec1051b766e184492fafa05d1b20a6ec", element="fa8a9512-074c-4aa7-89d4-0b03b41255a5")>]

In [19]:
initial_page = pages[0]
initial_page.text

'2'

In [20]:
final_page = pages[1]
final_page.text

'37'

In [26]:

    # Get the url of the website (parameter)
    pagination01 = config()['websites']['admin_users']['labels']['page01']['url_pagination01']
    pagination02 = config()['websites']['admin_users']['labels']['page01']['url_pagination02']
    pagination = pagination01 + "0" + pagination02 + "1"
    pagination

'sort.do?layoutCollection=0&layoutCollectionProperty=&layoutCollectionState=0&pagerPage=1'

In [ ]:
# Change to page 2 and so on...
# ...get the link to next page

initial_page.get_attribute('href')

In [ ]:
# Change to next page

initial_page.click()


In [ ]:
# Count the pages...
class_td = config()['websites']['admin_users']['labels']['page05']['class_td']
class_a = config()['websites']['admin_users']['labels']['page05']['class_a']
pages = driver.find_element_by_xpath('//td[@class="' + class_td + '"]').find_elements_by_xpath('//a[@class="' + class_a + '"]')
len(pages)


In [ ]:
print(pages[0].text)
print(pages[1].text)
print(pages[2].text)

In [ ]:
pages[1].click()

In [ ]:
# Count the pages...
class_td = config()['websites']['admin_users']['labels']['page05']['class_td']
class_a = config()['websites']['admin_users']['labels']['page05']['class_a']
pages = driver.find_element_by_xpath('//td[@class="' + class_td + '"]').find_elements_by_xpath('//a[@class="' + class_a + '"]')
len(pages)

In [ ]:
print(pages[0].text)
print(pages[1].text)
print(pages[2].text)
print(pages[3].text)

In [ ]:
pages[2].click()

In [ ]:
# Again, select all the data in the same object with:

all_rows = driver.find_elements_by_xpath('//table[@class="' + class_table + '"]/tbody/tr/td/table/tbody/tr[not(@valign)]/td[contains(@class, "' + class_td + '")]')

len(all_rows)

In [ ]:
# Testing with the first and second rows
i = 0

print("Record #", int(i / 5 + 1))
curp = all_rows[i].text
print('CURP:', curp)
i = i + 1
matricula = all_rows[i].text
print('Matrícula:', matricula)
i = i + 1
nombre = all_rows[i].text
print('Nombre:', nombre)
i = i + 1
cuenta = all_rows[i].text
print('Cuenta:', cuenta)
i = i + 1
grupo = all_rows[i].text
print('Grupo:', grupo)
i = i + 1
print("")

print("Record #", int(i / 5 + 1))
curp = all_rows[i].text
print('CURP:', curp)
i = i + 1
matricula = all_rows[i].text
print('Matrícula:', matricula)
i = i + 1
nombre = all_rows[i].text
print('Nombre:', nombre)
i = i + 1
cuenta = all_rows[i].text
print('Cuenta:', cuenta)
i = i + 1
grupo = all_rows[i].text
print('Grupo:', grupo)
i = i + 1

### It works!